# Big data lab

## import library

In [68]:
import pandas as pd
import pickle
data_path = "data-from-BLD/churn_model.pcl"
df = pd.read_pickle(data_path)

In [52]:
import pickle
from datetime import datetime, timedelta

def save_model(model = None, features = []):
    name = str(model.__class__).split('.')[-1][:-2] + '_' + datetime.today().strftime("%d%m%Y_%H_%M") + '.pickle'    
    if model:
        with open(name, 'wb') as file:
            pickle.dump((model, features), file)
            print('Save', name)
def load(file_name):
    with open(file_name, 'rb') as file:
        model_tpl = pickle.load(file)
    return model_tpl  

In [71]:
type(df[1])

tuple

In [53]:
df1 = load(data_path)

In [67]:
df1[1]

({'1': 'TRAIN_MODEL_FE', '2': 'TRAIN_MODEL_B_NUM', '3': 'TRAIN_MODEL_DPI'},
         Ama_rchrgmnt_sum_max_mnt1  content_clc_mea_mnt1  content_cnt_max_mnt1  \
 0                               0                   0.0             13.841608   
 1                               0                   0.0             11.361162   
 2                               0                   0.0             10.269117   
 3                               0                   0.0              9.974412   
 4                               0                   0.0              6.749902   
 ...                           ...                   ...                   ...   
 149995                          0                   0.0              8.333536   
 149996                          0                   0.0             14.038728   
 149997                          0                   0.0             13.225565   
 149998                          0                   0.0             19.524976   
 149999               

In [66]:
df2

({'1': 'TRAIN_MODEL_FE', '2': 'TRAIN_MODEL_B_NUM', '3': 'TRAIN_MODEL_DPI'},
         Ama_rchrgmnt_sum_max_mnt1  content_clc_mea_mnt1  content_cnt_max_mnt1  \
 0                               0                   0.0             13.841608   
 1                               0                   0.0             11.361162   
 2                               0                   0.0             10.269117   
 3                               0                   0.0              9.974412   
 4                               0                   0.0              6.749902   
 ...                           ...                   ...                   ...   
 149995                          0                   0.0              8.333536   
 149996                          0                   0.0             14.038728   
 149997                          0                   0.0             13.225565   
 149998                          0                   0.0             19.524976   
 149999               